In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs=[WebBaseLoader(url).load() for url in urls]
doc_list=[item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=50, chunk_overlap=0)

doc_splits=text_splitter.split_documents(doc_list)

embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")

vectorstore=Chroma.from_documents(doc_splits, embeddings,collection_name="rag-chroma")

retriever=vectorstore.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\shawn\AppData\Local\Temp\ipykernel_8980\3492377944.py:19: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")
c:\Users\shawn\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel,Field

class GradeDocuments(BaseModel):

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no' "
    )


In [10]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model="llama-3.1-70b-versatile"
)

In [11]:
structured_llm_grader=llm.with_structured_output(GradeDocuments)
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader=grade_prompt | structured_llm_grader
question="agent_memory"
docs=retriever.get_relevant_documents(question)
doc_text=docs[1].page_content
print(retrieval_grader.invoke({"document": doc_text, "question": question}))

C:\Users\shawn\AppData\Local\Temp\ipykernel_8980\3332864624.py:14: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs=retriever.get_relevant_documents(question)


binary_score='yes'


In [18]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
prompt = hub.pull("rlm/rag-prompt")

def format_dosc(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain=prompt | llm | StrOutputParser()
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

c:\Users\shawn\AppData\Local\Programs\Python\Python312\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In a LLM-powered autonomous agent system, there are two types of memory: short-term memory and long-term memory. Short-term memory utilizes in-context learning, while long-term memory provides the capability to retain and recall information over extended periods, often by leveraging an external vector store and fast retrieval. The agent's memory module is designed to record a comprehensive list of agents' experience in natural language.


In [19]:
system = """You a question re-writer that converts an input question to a better version that is optimized \n 
     for web search. Look at the input and try to reason about the underlying semantic intent / meaning.
     Just output the rewritten question, do not add any other text. \n\n"""
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial question: \n\n {question} \n Formulate an improved question.",
        ),
    ]
)
question_rewriter=re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

'What is agent memory in artificial intelligence and how does it work?'

In [41]:
from langchain_community.tools import DuckDuckGoSearchResults
web_search_tool=DuckDuckGoSearchResults(max_results=3)
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
wikipedia_tool= WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())


In [21]:
from typing import List
from typing_extensions import TypedDict

class GraphState(TypedDict):
    question: str
    generation: str
    web_search: str
    documents: List[str]

In [48]:
from langchain.schema import Document

def retrieve(state):
    print('RETRIEVE')
    question=state["question"]
    documents=retriever.get_relevant_documents(question)
    return {"documents": documents,"question": question}

def generate(state):
    print('GENERATE')
    question=state["question"]
    documents=state["documents"]
    generation=rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}

def grade_documents(state):

    print('CHECK RELEVANCE')
    question=state["question"]
    documents=state["documents"]
    
    filtered_docs=[]
    web_search="no"
    for d in documents:
        score=retrieval_grader.invoke({"document": d.page_content, "question": question})
        grade=score.binary_score
        if grade=="yes":
            print('DOCUMENT IS RELEVANT')
            filtered_docs.append(d)
        else:
            print('DOCUMENT IS NOT RELEVANT')
            web_search="yes"
            continue
    return {"documents": filtered_docs, "question": question, "web_search": web_search}



def transform_query(state):
    print('TRANSFORM QUERY')
    question=state["question"]
    documents=state["documents"]
    new_question=question_rewriter.invoke({"question": question})
    return {"documents": state["documents"], "question": new_question}


def web_search(state):
    print('WEB SEARCH')
    question=state["question"]
    documents=state["documents"]
    search_results=wikipedia_tool.run(question)
    web_results=Document(page_content=search_results)
    documents.append(web_results)
    return {"documents": documents, "question": question}


def decide_to_generate(state):
    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    web_search = state["web_search"]
    state["documents"]

    if web_search == "yes":
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---"
        )
        return "transform_query"
    else:
        print("---DECISION: GENERATE---")
        return "generate"

In [49]:
from langgraph.graph import START,END,StateGraph

workflow=StateGraph(GraphState)

workflow.add_node("retriever",retrieve)
workflow.add_node("grader",grade_documents)
workflow.add_node("generate",generate)
workflow.add_node("transform_query",transform_query)
workflow.add_node("web_search_node",web_search)

workflow.add_edge(START,"retriever")
workflow.add_edge("retriever", "grader")
workflow.add_conditional_edges("grader",decide_to_generate,{
        "transform_query": "transform_query",
        "generate": "generate",
    })
workflow.add_edge("transform_query", "web_search_node")
workflow.add_edge("web_search_node", "generate")
workflow.add_edge("generate", END)

app=workflow.compile()


In [51]:
from pprint import pprint

# Run
inputs = {"question": "What is AI"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Node '{key}':")
    pprint("\n---\n")

# Final generation
pprint(value["generation"])

RETRIEVE
"Node 'retriever':"
'\n---\n'
CHECK RELEVANCE
DOCUMENT IS RELEVANT
DOCUMENT IS RELEVANT
DOCUMENT IS RELEVANT
DOCUMENT IS RELEVANT
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
"Node 'grader':"
'\n---\n'
GENERATE
"Node 'generate':"
'\n---\n'
('AI, or Artificial Intelligence, refers to a powerful general problem solver '
 'that can be framed as a brain in an autonomous agent system, utilizing '
 'components like planning, memory, and tool use to handle complex tasks. It '
 'can generate well-written copies, stories, essays, and programs, and can '
 'also learn from mistakes and refine its actions. The provided context does '
 'not give a comprehensive definition of AI, but it highlights its potential '
 'and applications in LLM-powered autonomous agent systems.')
